In [11]:
import pandas as pd

# load data into pandas dataframe
df = pd.read_csv('../data/input/Salaries.csv')

# Set max num of rows to display
pd.set_option('display.max_rows', 1500)

df.head()

C:\Users\Hamz\AppData\Local\Temp\ipykernel_1632\1422585331.py:4: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/input/Salaries.csv')


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


## Cleaning

In [12]:
# check info about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148654 non-null  int64  
 1   EmployeeName      148654 non-null  object 
 2   JobTitle          148654 non-null  object 
 3   BasePay           148049 non-null  object 
 4   OvertimePay       148654 non-null  object 
 5   OtherPay          148654 non-null  object 
 6   Benefits          112495 non-null  object 
 7   TotalPay          148654 non-null  float64
 8   TotalPayBenefits  148654 non-null  float64
 9   Year              148654 non-null  int64  
 10  Notes             0 non-null       float64
 11  Agency            148654 non-null  object 
 12  Status            38119 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 14.7+ MB


The info tells us that all columns are object type although a lot of them are numeric columns. There are missing values in this dataset that are represented by empty strings, we can change both of these at the same time:

In [13]:
# convert Id and Year columns to int dtype
df[['Id', 'Year']] = df[['Id', 'Year']].astype(int)

# convert multiple columns to float, "errors='coerce'" converts any non-numeric values to NaN
float_columns = ['BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits']
for col in float_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Id                148654 non-null  int32  
 1   EmployeeName      148654 non-null  object 
 2   JobTitle          148654 non-null  object 
 3   BasePay           148045 non-null  float64
 4   OvertimePay       148650 non-null  float64
 5   OtherPay          148650 non-null  float64
 6   Benefits          112491 non-null  float64
 7   TotalPay          148654 non-null  float64
 8   TotalPayBenefits  148654 non-null  float64
 9   Year              148654 non-null  int32  
 10  Notes             0 non-null       float64
 11  Agency            148654 non-null  object 
 12  Status            38119 non-null   object 
dtypes: float64(7), int32(2), object(4)
memory usage: 13.6+ MB


In [15]:
df.isnull().sum()

Id                       0
EmployeeName             0
JobTitle                 0
BasePay                609
OvertimePay              4
OtherPay                 4
Benefits             36163
TotalPay                 0
TotalPayBenefits         0
Year                     0
Notes               148654
Agency                   0
Status              110535
dtype: int64

In [16]:
# dropping columns: 'Agency' is a zero variance column (only San Francisco), 'Notes' column has 100% missing values, and 'Status' has 74% missing values, checking the dataset on Kaggle.
df.drop(['Agency', 'Notes', 'Status'], axis=1, inplace=True)

In [17]:
# drop basepay missing rows (only 609, 0.4%)
df.dropna(subset=['BasePay'], inplace=True)

'Benefits' has 36159 missing values. I will filter the rows that have missing 'Benefits' and where 'TotalPay' and 'TotalPayBenefits' have the same value, and then see if there is also 36159 entries. If it does, I can set all missing 'Benefits' values to 0 since it means there is no benefits for these entries.

In [18]:
# check number of missing entries that also have the same totalpay and totalpaybenefits values
missing_benefits_same_totalpay = df[(df['Benefits'].isna()) & (df['TotalPay'] == df['TotalPayBenefits'])]
missing_benefits_same_totalpay.shape

(36159, 10)

In [19]:
# change all null benefits to 0
df['Benefits'].fillna(0, inplace=True)

In [20]:
# check for duplicate entries
duplicates = df.duplicated()
num_duplicated = duplicates.sum()
print(f"There are {num_duplicated} duplicate rows in the df")

There are 0 duplicate rows in the df


In [21]:
df.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,0.0,567595.43,567595.43,2011
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,0.0,538909.28,538909.28,2011
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,0.0,335279.91,335279.91,2011
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,0.0,332343.61,332343.61,2011
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,0.0,326373.19,326373.19,2011


## Data Aggregation

In [22]:
# group by job title and calculate stats of basepay and totalpay (mean, median, max, standard deviation, count)
agg_df = df.groupby(['JobTitle']).agg({'BasePay': ['mean', 'median', 'min', 'max', 'std', 'count'], 'TotalPay': ['mean', 'median', 'min', 'max', 'std', 'count']}).reset_index()

In [23]:
agg_df.head(15)

JobTitle        BasePay   
                                                                mean   
0                                       ACCOUNT CLERK   43300.806506  \
1                                          ACCOUNTANT   46643.172000   
2                                   ACCOUNTANT INTERN   28732.663958   
3                         ACPO,JuvP, Juv Prob (SFERS)   62290.780000   
4                                       ACUPUNCTURIST   66374.400000   
5                              ADMINISTRATIVE ANALYST   63435.628602   
6                           ADMINISTRATIVE ANALYST II   59373.710000   
7                          ADMINISTRATIVE ANALYST III   92698.515000   
8                             ADMINISTRATIVE ENGINEER  120913.161000   
9                     ADMINISTRATIVE SERVICES MANAGER   75555.033333   
10                 ADMINISTRATOR, SFGH MEDICAL CENTER  245124.440000   
11                      AFFIRMATIVE ACTION SPECIALIST   68177.316667   
12                             AGRICULTURAL INSPECTOR   53274.100000   
13  AIRPORT ASSISTANT DEPUTY DIRECTOR, BUSINESS AD...    1927.500000   
14      AIRPORT ASSISTANT DEPUTY DIRECTOR, OPERATIONS   15420.000000   

                                                               TotalPay   
        median        min        max           std count           mean   
0    49777.780     614.00   60838.20  12890.631740    83   44035.664337  \
1    60076.240       0.00   65392.01  27703.019579     5   47429.268000   
2    16806.755    2981.53   58501.72  20635.668379    48   29031.742917   
3    62290.780   62290.78   62290.78           NaN     1   62290.780000   
4    66374.400   66374.40   66374.40           NaN     1   67594.400000   
5    71602.900       0.00   78587.89  19079.519592    93   63954.867527   
6    59373.710   43161.60   75585.82  22927.385837     2   61497.035000   
7    92698.515   92550.03   92847.00    209.989501     2   92698.515000   
8   130171.010   72259.20  136679.61  19537.388445    10  127961.430000   
9    84783.370   49879.73   92002.00  22526.496747     3   77015.580000   
10  245124.440  245124.44  245124.44           NaN     1  257124.440000   
11   79511.680   13872.50   79696.00  26628.236215     6   68213.983333   
12   53274.100   48787.60   57760.60   6344.869148     2   53586.100000   
13    1927.500    1927.50    1927.50           NaN     1    1927.500000   
14   15420.000   15420.00   15420.00           NaN     1   15420.000000   

                                                          
        median        min        max           std count  
0    49777.820     614.00   60838.20  13170.742908    83  
1    60076.240    1148.40   65392.01  27137.331315     5  
2    17658.505    2981.53   58799.53  20711.322649    48  
3    62290.780   62290.78   62290.78           NaN     1  
4    67594.400   67594.40   67594.40           NaN     1  
5    71602.900    2388.24   79754.03  18613.695839    93  
6    61497.035   47408.25   75585.82  19924.550824     2  
7    92698.515   92550.03   92847.00    209.989501     2  
8   130171.065   91833.33  149322.23  14993.074838    10  
9    84783.370   54261.37   92002.00  20033.539610     3  
10  257124.440  257124.44  257124.44           NaN     1  
11   79621.680   13872.50   79696.00  26646.805332     6  
12   53586.100   48787.60   58384.60   6786.103779     2  
13    1927.500    1927.50    1927.50           NaN     1  
14   15420.000   15420.00   15420.00           NaN     1

## Data Engineering
I am going to create 2 new features. The first feature (JobCategory) will categorise job titles. For example, 'law enforcement' will contain police and similar roles. The second feature (SalaryRangeCategory) will categorise different salary ranges, such as low, medium and high salaries depending on the minimum, maximum and mean salary for that job title. 

In [ ]:
import re

# define regex patterns for each category
job_categories = {
    'Law Enforcement' : ['police', 'sheriff', 'crime', 'forensic(s)?', 'patrol', 'detective', 'mayor', 'sergeant', 'captain', 'officer', 'lieutenant'],
    'Safety & Security' : ['fire', 'safety', 'public', 'security', 'guard', 'protect(ive)?'],
    'Medical' : ['doctor', 'nurse', 'paramedic', 'medic(al)?', 'health(care)?', 'medicine', 'anesthetist'],
    'Education' : ['teacher', 'prof(essor)?', 'teacher(s)? assistant', 'education', 'eng(r)?'],
    'Administrative' : ['clerk', 'admin', 'secretary', 'assistant', 'library', 'librarian'],
    'Engineering' : ['engineer', 'architect', 'technician', 'physician', r'\belectr\w+'],
    'Construction' : ['construction', 'mechanic', 'laborer'], 
    'Information Technology' : ['programmer', 'developer', 'software', 'IT', 'computer', 'analyst'],
    'Management' : ['manager', 'director', 'CEO', 'owner', 'supervisor', 'head', 'leader'],
    'Finance' : ['accountant', 'economist', 'tax', 'finance', 'money'],
    'Legal' : ['law(yer)?', 'legal', 'attorney', 'judge'],
    'Maintenance' : ['custodian', 'porter', 'gardener'], 
    'Other' : []   
}

# function to assign job categories based on regex patterns
def categorise_job_title(job_title):
    for category, patterns in job_categories.items():
        for pattern in patterns:
            if re.search(pattern, job_title, re.IGNORECASE):
                return category
    return 'Other'

# apply the categorise_job_title function to the JobTitle column and create new column
df['JobCategory'] = df['JobTitle'].apply(categorise_job_title)

In [ ]:
df.head()

In [ ]:
# the second feature will compare TotalPay to the std for that job title and categorise it (low, medium, high)
# create empty column
df['SalaryRangeCategory'] = ''

# merge the aggregated df and original df to compare salaries for job titles
agg_df.columns = ['JobTitle', 'BasePay_mean', 'BasePay_median', 'BasePay_min', 'BasePay_max', 'BasePay_std', 'BasePay_count', 'TotalPay_mean', 'TotalPay_median', 'TotalPay_min', 'TotalPay_max', 'TotalPay_std', 'TotalPay_count']
merged_df = df.merge(agg_df, on='JobTitle')

In [ ]:
# calculate the TotalPay salary range for each row, if pay is less than 1 std below mean, salary range will be low. Above will be high and within will be medium.
for i, row in merged_df.iterrows():
    mean = row['TotalPay_mean']
    std = row['TotalPay_std']
    salary = row['TotalPay']
    if salary < mean - std:
        df.at[i, 'SalaryRangeCategory'] = 'low'
    elif salary > mean + std:
        df.at[i, 'SalaryRangeCategory'] = 'high'
    else:
        df.at[i, 'SalaryRangeCategory'] = 'medium'
        
# fill in missing values (caused by std = NaN) with 'unknown'
df['SalaryRangeCategory'] = df['SalaryRangeCategory'].fillna('unknown')

In [ ]:
# transforming the dataset is finished. Display both agg_df and df:
df.head()

In [ ]:
agg_df.head()